# (Technical) Jupyter Notebook Standardizing Identifier Order In Adjacent Columns in hu.MAP3-provided CSV

Ah, classic data wrangling story....

Long story, short, I had assumed in the author-provided CSV the indentifiers listed in the **adjacent** comma-separated column Uniprot_ACCs and column genenames were an in-order match. **They were not.** This effort fixes that in the end because it seems like a good data managemet practice and makes steps in my pipelines easier. Along the way I check maybe what was going on out of curiousity.

------

**Details**, i.e., the long story:

Authors provided a CSV about the hu.MAP 3.0 complex. (See below where I remark, 'I had wanted to get the notebook every time from the source...' for how I had wanted to get that direct for all the Jupyter Notebooks in this related series.)

Looking at a few examples I had assumed the **adjacent** comma-separated column Uniprot_ACCs and column genenames were an in-order match. Let's look at some raw rows of the data to see if that was valid I assumed that.

Example from the raw data (line 6663):

```text
huMAP3_06662.1,2,O14832 O75381 Q8WY41,PHYH PEX14 NANOS1
```

- [O14832](https://www.uniprot.org/uniprotkb/O14832/entry) corresponds to `PHYH`.
- [O75381](https://www.uniprot.org/uniprotkb/O75381/entry) corresponds to `PEX14`.
- [Q8WY41](https://www.uniprot.org/uniprotkb/Q8WY41/entry) correspinds to `NANOS`.

So they are an in-order match in the two columns.  
(There's more examples below,under 'Detour to see why mismatching number of identifiers in the two columns, `Uniprot_ACCs` & `genenames`, in 279 rows', where the order seems to match in the order in the two colums.)
And that seems good data practice and so I assumed they matched.  
WRONG!!!

Looking at NHP2 example (line 6379):

```text
huMAP3_06378.1,2,Q8N8A6 Q9NX24 Q9Y2R4,NHP2 DDX52 DDX51
```

It turns out there:
- [Q8N8A6](https://www.uniprot.org/uniprotkb/Q8N8A6/entry) corresponds to `DDX51`.
- [Q9Y2R4](https://www.uniprot.org/uniprotkb/Q9Y2R4/entry) corresponds to `DDX52`.
- [Q9NX24](https://www.uniprot.org/uniprotkb/Q9NX24/entry) corresponds to `NHP2`.

And so the order doesn't seem to match at all!?!?! The column after the comma doesn't match; teh right column would need to be DDX51, NHP2, DDX52 to match leftc column. (See me questioning what is alphabetical here just below.)

That caused issues downstream with extra versions and things not matching appropriately between accession and gene names when I filtered for a unique set as I was using the order to match them up.

**This effort fixes/validates that to produce a standardized matching order for the identifiers in the two columns in the end.**   
I was already getting the data not from the source  (see below where I remark, 'I had wanted to get the notebook every time from the source...'). So I might as well make it more standardized and easier to use downstream for myself and others.
A few reasons rationalizing this:
- the step to make the look-up is time consuming (have to look up and sort UniProt KB info all 13769 identifiers and not just the subset complexed with your protein of inters) and so it would have been a pain to fix at the start of a new session everytime.
- I had already written code in a few places that assumes an in-order match between the identifiers in the two adjacent columns and so since I have to get the data not from the source, already might was well make it more useable. 
- standardized in-order match between the data in the two adjacent columns just seems like better data management practice in general.

Out of curiosity, along the way to fixing/standardizing I want to see if the sequence of the identifiers not matching was there all along just by chance sampling a small example I happened to think they were in matching order sequence? ..... Was it just data transformation steps gone wrong? .... Are they in alphantical order within each column maybe? .... Corresponding matching order would make more sense for the adjacent but maybe they used Excel along the way it and it transformed things in the process?

For the notebooks in this related series stored here in my humap3-binder repo, I had wanted to get the notebook every time from the source so always can say based on what was prvoided but MyBinder seems to block the involved port (maybe?) because for whatever reason I cannot `curl` getting that file from the hu.MAP3 source and so I was storing it in [a gist here](https://gist.githubusercontent.com/fomightez/d84614a9250f13b8d24217b1330c66e3) where MyBinder could access via curl. Turned out in the long run that wouldn't have worked very well anyway because would need time-consuming step of collecting information about identifiers and fixing the adjacent columns so identifiers in column Uniprot_ACCs and column genenames were an in-order match everytime. So I'm just going to fix. This document will also serve as my way to record these steps to fix too, and in that way if the investigatos release an update or a highly related CSV file, I can run this again to validate or fix as well.

#### Quick check, is it alphabetical?

In [ ]:
huMAP3_06137.1,2,Q8N475 Q9P2F8 Q9Y250,LZTS1 FSTL5 SIPA1L2

In [5]:
#raw data Q8N8A6 Q9NX24 Q9Y2R4,NHP2 DDX52 DDX51
mylist = ["Q8N8A6", "Q9NX24", "Q9Y2R4"]
mylist.sort()
mylist # alphabetical: `['Q8N8A6', 'Q9NX24', 'Q9Y2R4']` and so those accessions in data seem alphabetical; other side would neex to be DDX51, NHP2, DDX52 to match

['Q8N8A6', 'Q9NX24', 'Q9Y2R4']

In [6]:
mylist = ["NHP2", "DDX52", "DDX51"]
mylist.sort()
mylist # Looks like alphabetical here would be ['DDX51', 'DDX52', 'NHP2'] so they aren't alphabetical under `genenames` themselves, not clear at all why `genenames` in that order

['DDX51', 'DDX52', 'NHP2']

- [Q8N8A6](https://www.uniprot.org/uniprotkb/Q8N8A6/entry) corresponds to `DDX51`.
- [Q9Y2R4](https://www.uniprot.org/uniprotkb/Q9Y2R4/entry) corresponds to `DDX52`.
- [Q9NX24](https://www.uniprot.org/uniprotkb/Q9NX24/entry) corresponds to `NHP2`.

-------

### Preparation


##### Get the complexes with confidence scores

While `curl -OL "https://humap3.proteincomplexes.org/static/downloads/humap3/hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922.csv"` works on my local machine, the involved port may be blocked on MyBinder for getting it from the original resource.  
I suspect this won't change much and so for now I am getitng it from a copy I made from the original source. So if you find something interesting & want to confirm, get the newest data and replace this file by drag and dropping from your local machine into the file navigation paenl on the right. and check.

In [1]:
!curl -OL https://gist.githubusercontent.com/fomightez/d84614a9250f13b8d24217b1330c66e3/raw/d2d6cb846ba22c1f9f3a4fb36002aa4ce9ba621f/hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1241k  100 1241k    0     0  2190k      0 --:--:-- --:--:-- --:--:-- 2189k


##### Put the data on the complexes into Pandas dataframe

(I'm using uv here just because I want to learn about it. I could have run the code in the script right in this notebook, and skipped the pickling and read pickle steps.)

Get the script to use with `uv` to read in the raw data and make a dataframe.

In [2]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3289      0 --:--:-- --:--:-- --:--:--  3290


In [6]:
!uv run complexes_rawCSV_to_df.py hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922.csv
import pandas as pd
rd_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
rd_df

Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               

,HuMAP3_ID,ComplexConfidence,Uniprot_ACCs,genenames
0,huMAP3_00000.1,1,P20963 Q9NWV4 Q9UGQ2,CD247 CACFD1 CZIB
1,huMAP3_00001.1,1,O94887 Q9NQ92 Q9NWB1,FARP2 COPRS RBFOX1
2,huMAP3_00002.1,1,Q8N3D4 Q9Y3A4,RRP7A EHBP1L1
3,huMAP3_00003.1,1,O00429 Q5T2D2,DNM1L TREML2
4,huMAP3_00004.1,1,O95460 P21941 P78540 Q9H267 Q9H9C1,MATN4 MATN1 ARG2 VPS33B VIPAS39
...,...,...,...,...
15321,huMAP3_15345.1,6,O14628 Q3SXZ3,ZNF195 ZNF718
15322,huMAP3_15346.1,6,P08910 Q6ZWT7 Q86VD1 Q9UJQ1 Q9Y6X9,ABHD2 MBOAT2 MORC1 LAMP5 MORC2
15323,huMAP3_15347.1,6,A6ND91 Q4V339,ZNG1F ASPDH
15324,huMAP3_15348.1,6,A6NKF2 P08217 Q8IVW6 Q99856,CELA2A ARID3B ARID3A ARID3C


Initial idea to make in-order was to collect the identifiers in the Uniprot_ACCs & genenames columns by exploding the columns **together** and then making a list that is reduced to just the unique set. Then that would serve as a basis making a lookup table to relate the contents of the two columns.    
But that doesn't work for both columns together as I had discovered, & cover next the reasoning, and so the code is left with what works and that is just 'exploding' one column:

In [4]:
# Can use Pandas `explode()`-related code I worked out in relation to using `rd_df` data already (see `notebooks/Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb`)
# to make the step of flattening the Uniprot_ACCs
intermed_df = rd_df.copy()
intermed_df['Uniprot_ACCs'] = intermed_df['Uniprot_ACCs'].str.split()
#intermed_df['genenames'] = intermed_df['genenames'].str.split() # cannot use explode with both columns because out of 15326 rows, 279 rows don't have same number of indentifiers in the two columns (?!?! Do some identifiers occur twice in one or other?)
# Now use explode to create a new row for each element in both columns
expanded_df = intermed_df.explode(['Uniprot_ACCs']).copy()
# Reset the index 
expanded_df = expanded_df.reset_index(drop=True)
len(expanded_df)

75531

That didn't work as I had thought for both columns, read on for as to why and how fixed so ultimately exploding those two columns together will work (since the code I made made elsewhere and used was assuming 'exploding' the columns would work well)....

#### Detour to see why mismatching number of identifiers in the two columns, `Uniprot_ACCs` & `genenames`, in 279 rows

Noted something curious while drafting the code for the above cell..

Found couldn't use `explode()` with both columns of the entire 'raw data' dataframe because out of 15326 rows, 279 rows (see next cell showing this) don't have same number of indentifiers in the two columns (?!?! Do some identifiers occur twice in one or other?)

In [5]:
intermed_df = rd_df.copy()
intermed_df['Uniprot_ACCs'] = intermed_df['Uniprot_ACCs'].str.split()
intermed_df['genenames'] = intermed_df['genenames'].str.split()
#intermed_df.head()
num = 0
for row in intermed_df.itertuples():
    if len(row.Uniprot_ACCs) != len(row.genenames):
        num+=1
import rich
rich.print(f"Total rows in the CSV: [bold black]{len(intermed_df)}[/bold black]")
rich.print(f"Total Rows where the number of identifiers in the\ntwo adjacent columns aren't the same: [bold black]{num}[/bold black]")

Total rows in the CSV: 15326

Total Rows where the number of identifiers in the
two adjacent columns aren't the same: 279

Show some examples that can be explored:

In [6]:
# Filter rows where list lengths differ
i_df = rd_df.copy()
i_df['Uniprot_ACCs'] = i_df['Uniprot_ACCs'].str.split()
i_df['genenames'] = i_df['genenames'].str.split()
mismtchd = i_df[i_df['Uniprot_ACCs'].apply(len) != i_df['genenames'].apply(len)]

# Randomly select 10 rows from the filtered DataFrame
sample_rows = mismtchd.sample(n=10, random_state=3) #use state to assure sampling is same when re-run
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(sample_rows)

,HuMAP3_ID,ComplexConfidence,Uniprot_ACCs,genenames
13129,huMAP3_13145.1,6,"[Q5TCX8, Q9BQ83]","[MAP3K21, SLX1A;, SLX1B]"
5650,huMAP3_05650.1,2,"[P78423, Q8IVW1, Q8NHU3]","[CX3CL1, SGMS2, ARL17A;, ARL17B]"
7576,huMAP3_07577.1,2,"[P68431, Q8N806]","[H3C1;, H3C2;, H3C3;, H3C4;, H3C6;, H3C7;, H3C8;, H3C10;, H3C11;, H3C12, UBR7]"
6588,huMAP3_06589.1,2,"[A0A087X2D5, B2CML4, O14561, O75394, O95900, P09001, P49406, P52815, P79522, P82663, Q13084, Q13405, Q13595, Q14197, Q15070, Q15434, Q16540, Q1KMD3, Q4U2R6, Q5T653, Q6P087, Q6P161, Q6P1L8, Q6XE24, Q7Z7F7, Q7Z7H8, Q86TS9, Q8IUH3, Q8IXM3, Q8IYB8, Q8N0T1, Q8N5N7, Q8N983, Q8TAE8, Q8TCC3, Q969S9, Q96A35, Q96CM3, Q96DH6, Q96DV4, Q96EH3, Q96EL3, Q96GC5, Q96I24, Q96KR1, Q96SI9, Q9BQ48, Q9BQC6, Q9BYC8, Q9BYC9, Q9BYD1, Q9BYD2, Q9BYD3, Q9BYD6, Q9BYK8, Q9BZE1, Q9H0U6, Q9H2W6, Q9H9J2, Q9HC36, Q9HD33, Q9NP92, Q9NPE2, Q9NQ50, Q9NRX2, Q9NUL7, Q9NVS2, Q9NWU5, Q9NX20, Q9NXV6, Q9NYK5, Q9NYY8, Q9NZE8, Q9P015, Q9P0M9, Q9UBX3, Q9Y2Z4, Q9Y3B7, Q9Y6G3]","[NDUFAB1, TRUB2, MRPL12, PRR3, MRPL28, MRPL49, TRA2A, MRPL58, OXA1L, MRPL23, RPUSD3, RBMS3, MRPL10, MRPL52, RBM45, MRPL41, SUPV3L1, MRPL43, GADD45GIP1, GFM2, RPUSD4, MSI2, MRPL38, MALSU1, MRPL53, FUBP3, STRBP, HELZ2, MRPL18, MRPL44, MRM3, NGRN, MRPL40, DDX28, MRPL22, CDKN2AIP, MRPL39, FASTKD2, MRPL35, SLC25A10, YARS2, MRPL11, MRPL42, MRPL33, MRPL3, MRPL19, MRPS25, MRPL51, MRPL2, MRPL14, MRPL55, RBIS, MRPL50, MRPL30, MRPL24, MRPL48, ZFR, MRPL57, MRPL32, MRPL20, MRPL13, MRPL9, MRPL4, MRPL37, MRPL46, MRPL47, MRPS30, MRPL17, MRPS18A, MRPL16, MRPL15, MRPL27, RBMS2, HNRNPUL2, MRPL54, MRPL34, MRPL1, MRPL45]"
6035,huMAP3_06036.1,2,"[P07357, P32246, P49441, Q71DI3, Q9H6B9, Q9H720]","[C8A, CCR1, H3C15;, H3C14;, H3C13, EPHX3, INPP1, CWH43]"
6271,huMAP3_06272.1,2,"[Q8WV35, Q9NZ71]",[RTEL1]
12397,huMAP3_12412.1,6,"[P28331, Q96IZ6, Q96LI6, Q96NC0, Q9NUJ1]","[NDUFS1, METTL2A, HSFY1;, HSFY2, ZMAT2, ABHD10]"
9321,huMAP3_09326.1,4,"[P49247, Q86VZ2, Q8NHG8, Q96JG8]","[RPIA, ZNRF2, MAGED4;, MAGED4B, WDR5B]"
14563,huMAP3_14582.1,6,"[645345, O75596, Q8NDY6]","[CLEC3A, BHLHE23]"
1702,huMAP3_01702.1,1,"[Q96PE3, Q9GZY0, Q9H4D5, Q9NPJ8]","[INPP4A, NXF2;, NXF2B, NXF3, NXT2]"


This is a nice sampling of examples to examine.

Let's look at a few:

Example from row #14563: `[645345, O75596, Q8NDY6]	[CLEC3A, BHLHE23]`

That first UniProt accession, `645345`, doesn't even match the format of typical UniProt accension. (`O75596` corresponds to `CLEC3A` and `Q8NDY6` corresponds to `BHLHE23`.) So it makes it hard to to tell what may be going on here since the two out of three under 'Uniprot_ACCs' are both accounted for in the genenames column. (I even looked at what the other 11 of 12 rows where `645345` occurs and I couldn't account for it from the rows where there was a manageable number to try and deconvolute the situation.)   
UPDATE: by searching associated proteins at the later I found [the NCBI link for this](https://www.ncbi.nlm.nih.gov/gene/645345) (the UniProt link they provided was a dead end) at the authors'-provided site for querying the complexes non-programmatically. That site links to [the HGNC enetry for this protein/gene](https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:23676) that has a UniProt entry, [Q5T1J5](https://www.uniprot.org/uniprotkb/Q5T1J5/entry). So why not use [Q5T1J5](https://www.uniprot.org/uniprotkb/Q5T1J5/entry) to be consistent with all the other **13768** protein identifiers in that column? Handled differently because putative?

Example from row #6271: `[Q8WV35, Q9NZ71]	[RTEL1]`
If I look up `RTEL1` at UniProt, I see [Q9NZ71](https://www.uniprot.org/uniprotkb/Q9NZ71/entry).
If I search `Q8WV35` at UniProt, I get [this](https://www.uniprot.org/uniprotkb/Q8WV35/history) where says:

```text
This entry is no longer annotated in UniProtKB and can be found in UniParc.
Reason: Deleted from Swiss-Prot
Since release: 2022_05/2022_05
```

(There's also an 'i' information indicating symbol after the `UniParc` text there that leads to a pop-up, see 'the information in the info-pop up' below.) 
Below is a table with the last entry 'Release Date' column at the top from 03-Aug-2022 and if I click `txt` next to `153` below 'Entry Version' column (first column) along that row, I end up at [here](
https://rest.uniprot.org/unisave/Q8WV35?format=txt&versions=153) where it has what looks to be a GENBANK like entry with the second line `AC   Q8WV35; B2RE92; Q9UKA0;` and the tenth line `GN   Name=LRRC29; Synonyms=FBL9, FBXL9;`

Searching that 'LRRC29' or 'FBXL9' in HGNC gets me to [here](https://genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:13605) where it says FBXL9P is a pseudogene.

That is consistent with the information in the info-pop up after UniParc that I mentioned above that says:

>"Deleted entries in UniProtKB/Swiss-Prot are mostly Open Reading Frames (ORFs) or pseudogenes that have been wrongly predicted to code for proteins. .... Most UniProtKB/TrEMBL deletions are due to the deletion of the corresponding coding sequence (CDS) in the source nucleotide sequence databases EMBL-Bank/DDBJ/GenBank as requested by the original submitters, or due to the deletion of the sequence prediction from Ensembl or RefSeq. ... In addition, some protein sequences are recognized by curators to be Open Reading frames (ORFs) that have been wrongly predicted to code for proteins or to be pseudogenes. When there is enough evidence that these hypothetical proteins are not real, we take the decision to remove them from UniProtKB/TrEMBL."

I guess the hu.MAP 3.0 data would argue it shouldn't have been deleted and this is the way they have of still including it, but why not in genenames?

So that would mean most would be expected to have more under Uniprot_ACCs if that shows psuedogenes and that general trend does seem to be the case....
But then what about example from row #13129: `[Q5TCX8, Q9BQ83]	[MAP3K21, SLX1A;, SLX1B]`
[Q5TCX8](https://www.uniprot.org/uniprotkb/Q5TCX8/entry) corresponds to `MAP3K21` and [Q9BQ83](https://www.uniprot.org/uniprotkb/Q9BQ83/entry) corresponds to `SLX1A; SLX1B`. I get why that one doesn't match. In that case the semi-colon is there to signify it has two names. (Keep in mind the original row there in the data CSV file looks like `huMAP3_13145.1,6,Q5TCX8 Q9BQ83,MAP3K21 SLX1A; SLX1B` because what is represented here is a Python list made from that content.)

So for row #12397 there is a case with a semi-colon, too:

```text
[P28331, Q96IZ6, Q96LI6, Q96NC0, Q9NUJ1]	[NDUFS1, METTL2A, HSFY1;, HSFY2, ZMAT2, ABHD10]
```

Does the way of writing the two name gene account for that there? Accounting for cheking this:

- [P28331](https://www.uniprot.org/uniprotkb/P28331/entry) corresponds to `NDUFS1`.
- [Q96IZ6](https://www.uniprot.org/uniprotkb/Q96IZ6/entry) corresponds to `METTL2A`.
- [Q96LI6](https://www.uniprot.org/uniprotkb/Q96LI6/entry) corresponds to `HSFY1; HSFY2`.

So the semi-colons seem to explain where more identifiers on the side of the `genenames` column.  
Later found at least two cases where there's three genenames, like `F8A1;F8A2;F8A3` and `TEX28;TEX28P1;TEX28P2`, per UniProt identifier.

Related observational take-away from looking at these cases that relates to below:  
The ones I investigated here, the position of the identifier on the `Uniprot_ACCs` column corresponds to the order in the `genenames` colunn. So this would argue for in-order matched and maybe I wasn't offbase to conclude that earlier. (But I have learned I definitely should have looked more intently!)

### Make lookup table for relating identifiers in the two columns, column Uniprot_ACCs and column genenames

Will look up the Uniprot_ACCs indentifiers in the UniProt Knowledgebase using the package Unipressed (see my [Unipressed-binder repo](https://github.com/fomightez/Unipressed-binder) for more on working with this package). Note that I will not use the ID mapping to HGNC because for one thing, `request = IdMappingClient.submit(source="UniProtKB_AC-ID", dest="HGNC", ids=id_list)` , gives results like `[{'from': 'Q5VV41', 'to': 'HGNC:15515'}]`. And so additional work would be needed to go to gene name there. Secondly, I know from looking, e.g., `SLX1A; SLX1B`. At HGNC, it is just `SLX1A` [there](https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:20922) and a separate entry for `SLX1B` [there](https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:28748), even though listed as same locus in both entries. Importantly, `SLX1B` is not on the `SLX1A` HGNC page, except I see `Slx1b` listed under moust ortholog, and so would be hard to expect to easily relate these types of things to what is used in the hu.MAP 3.0 data. I could though use `request = IdMappingClient.submit(source="UniProtKB_AC-ID", dest="GeneCards", ids=id_list)` and for `Q9BQ83` that returns two items: `[{'from': 'Q9BQ83', 'to': 'SLX1A'},{'from': 'Q9BQ83', 'to': 'SLX1B'}]`. However, while it doesn't return an HTTP error when you give it  'bad' ID, it discards anything where there is no match and so I miss being able to handle that as encountered.

First to get started creating that lookup, the next cell will make a list of all the unique identifiers in that column.

In [18]:
# first make a list of all the identifiers in the 'Uniprot_ACCs' column
ids_in_Uniprot_ACCs = list(set(expanded_df['Uniprot_ACCs'].to_list())) # the set will limit to unique set, eliminating ones that appear more than once
len(ids_in_Uniprot_ACCs)

13769

There's 13769 unique identifiers among all the identifiers in the 'Uniprot_ACCs' column. (If I do just over a second for each one to avoid slamming the site, that is 3 hours and 50 minutes. And so I should design this collecint to be able to restartable and do in chunks.)

Let's store that (or read back in, or essentially just skip over this cell, depending on situation) for getting back up and running during development optionally faster, without running all cells above again. (Otherwise, in the way the code is in the cell, the cell runs fine without doing anything in present form.)

In [6]:
# if need to read in the list of all 13769 uniue Uniprot_ACCs
'''
import pickle
with open("list_all_13769_uniue_Uniprot_ACCs.pkl", "rb") as f:
        ids_in_Uniprot_ACCs = pickle.load(f)
len(ids_in_Uniprot_ACCs)
'''

# if need to save the list of all 13769 uniue Uniprot_ACCs again for some reason
'''
import pickle
with open("list_all_13769_uniue_Uniprot_ACCs.pkl", "wb") as f:
        pickle.dump(ids_in_Uniprot_ACCs , f)
'''

In [2]:
import pickle
with open("list_all_13769_uniue_Uniprot_ACCs.pkl", "rb") as f:
        ids_in_Uniprot_ACCs = pickle.load(f)
len(ids_in_Uniprot_ACCs)

13769

Now with identifiers collected and in active memory, collect the gene names.

In [3]:
# This first cell sets things up so can run in chunks to progressively get to complete lookup dictionary eventually
from unipressed import UniprotkbClient
import time
import requests
import pickle
import os

def process_uniprot_chunk(id_list):
    """
    Process a chunk of UniProt IDs and return a dictionary of ID to gene name mappings.
    """
    chunk_dict = {}
    for uniprot_id in id_list:
        try:
            uniprot_record = UniprotkbClient.fetch_one(uniprot_id)
            #lookup_dict[uniprot_id] = uniprot_record['genes'][0]['geneName']['value'] #worked for when one 'gene' in the list returned by `uniprot_record['genes']`
            # HOWEVER...
            # some IDs like `Q96LI6` and `Q9BQ83` give more than one gene for `uniprot_record['genes']` and the CSV from the hu.MAP3 people was combining 
            # those to things like `HSFY1; HSFY2` and `SLX1A; SLX1B`, respectively. To do accurate accounting and keep close to that (or at least keep the option to keep close to that), I want to recapitulate that, too.
            if 'genes' in uniprot_record:
                chunk_dict[uniprot_id] = '; '.join([x['geneName']['value'] for x in uniprot_record['genes']])
            else:
                if uniprot_id == 'B3KT37': # special case I looked into
                    chunk_dict[uniprot_id] = 'SPECIALin_UniProt_and_VETELKLIC_part_similar_to_YWHAE_but_no_official_gene'
                else:
                    chunk_dict[uniprot_id] = 'SPECIALin_UniProt_but_no_gene'
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 400:
                chunk_dict[uniprot_id] = 'not_known'
                print(f"UniProt ID '{uniprot_id}' not found. Marked as 'not_known' in lookup_dict.\n(Although for known case, '645345', this will be fixed later.)")
            else:
                raise e
        time.sleep(1.12)
    return chunk_dict

def process_all_ids_in_chunks(id_list, chunk_size=1000, output_dir='chunk_pickles', resume=True):
    """
    Process all IDs in chunks and save each chunk as a pickle file.
    Supports resuming from the last completed chunk.
    
    Args:
        id_list (list): List of all UniProt IDs to process
        chunk_size (int): Size of each chunk
        output_dir (str): Directory to save pickle files
        resume (bool): If True, skip already processed chunks
    """
    os.makedirs(output_dir, exist_ok=True)
    
    total_chunks = (len(id_list) + chunk_size - 1) // chunk_size
    
    # Find the last completed chunk if resuming
    existing_chunks = []
    if resume:
        existing_chunks = [f for f in os.listdir(output_dir) if f.startswith('chunk_') and f.endswith('.pkl')]
        existing_chunks.sort()  # Sort to find the highest number
    
    # Determine starting chunk
    start_chunk = 0
    if existing_chunks:
        last_chunk = existing_chunks[-1]
        start_chunk = int(last_chunk.split('_')[1].split('.')[0])  # Extract number from 'chunk_003.pickle'
        print(f"Resuming from chunk {start_chunk + 1}")
    
    # Process remaining chunks
    for i in range(start_chunk * chunk_size, len(id_list), chunk_size):
        chunk_num = i // chunk_size + 1
        chunk = id_list[i:i + chunk_size]
        
        pickle_filename = os.path.join(output_dir, f'chunk_{chunk_num:03d}.pkl')
        
        # Skip if chunk already exists and we're resuming
        if resume and os.path.exists(pickle_filename):
            print(f"Skipping existing chunk {chunk_num}/{total_chunks}")
            continue
            
        print(f"Processing chunk {chunk_num}/{total_chunks} ({len(chunk)} IDs)")
        chunk_result = process_uniprot_chunk(chunk)
        
        with open(pickle_filename, 'wb') as f:
            pickle.dump(chunk_result, f)
        
        print(f"Saved {len(chunk_result)} entries to {pickle_filename}")

def combine_pickle_chunks(pickle_dir='chunk_pickles'):
    """
    Combine all pickle files in the directory into a single dictionary.
    
    Args:
        pickle_dir (str): Directory containing the pickle files
    
    Returns:
        dict: Combined dictionary of all chunks
    """
    combined_dict = {}
    pickle_files = sorted([f for f in os.listdir(pickle_dir) if f.endswith('.pkl')])
    
    for pickle_file in pickle_files:
        with open(os.path.join(pickle_dir, pickle_file), 'rb') as f:
            chunk_dict = pickle.load(f)
            combined_dict.update(chunk_dict)
        print(f"Loaded {pickle_file}, combined dictionary now has {len(combined_dict)} entries")
    
    return combined_dict

In [5]:
# This will take a LONG TIME TO run with all 13769 unique identifiers and so when developing use on a subset. Next line would be uncommented for that.
# When actually running, will do in chunks and CAN RESUME with initial chunks (as long as all you same `chunk_size` setting; see `ABOUT RESUMING:` below).
#ids_in_Uniprot_ACCs = ids_in_Uniprot_ACCs[:2999] #Only uncommented during development to limit to first ~3K!!
#ids_in_Uniprot_ACCs = ['Q96LI6'] # FOR DEBUGGING, example of one with two genenames
process_all_ids_in_chunks(ids_in_Uniprot_ACCs, chunk_size=500)  # ABOUT RESUMING: Can automatically resume from chunk 4 if you make a directory `chunk_pickles` in current working directory & drop in the first three there

Resuming from chunk 24
Processing chunk 24/28 (500 IDs)
Saved 500 entries to chunk_pickles/chunk_024.pkl
Processing chunk 25/28 (500 IDs)
Saved 500 entries to chunk_pickles/chunk_025.pkl
Processing chunk 26/28 (500 IDs)
Saved 500 entries to chunk_pickles/chunk_026.pkl
Processing chunk 27/28 (500 IDs)
Saved 500 entries to chunk_pickles/chunk_027.pkl
Processing chunk 28/28 (269 IDs)
Saved 269 entries to chunk_pickles/chunk_028.pkl


In [4]:
lookup_dict = combine_pickle_chunks()

Loaded chunk_001.pkl, combined dictionary now has 500 entries
Loaded chunk_002.pkl, combined dictionary now has 1000 entries
Loaded chunk_003.pkl, combined dictionary now has 1500 entries
Loaded chunk_004.pkl, combined dictionary now has 2000 entries
Loaded chunk_005.pkl, combined dictionary now has 2500 entries
Loaded chunk_006.pkl, combined dictionary now has 3000 entries
Loaded chunk_007.pkl, combined dictionary now has 3500 entries
Loaded chunk_008.pkl, combined dictionary now has 4000 entries
Loaded chunk_009.pkl, combined dictionary now has 4500 entries
Loaded chunk_010.pkl, combined dictionary now has 5000 entries
Loaded chunk_011.pkl, combined dictionary now has 5500 entries
Loaded chunk_012.pkl, combined dictionary now has 6000 entries
Loaded chunk_013.pkl, combined dictionary now has 6500 entries
Loaded chunk_014.pkl, combined dictionary now has 7000 entries
Loaded chunk_015.pkl, combined dictionary now has 7500 entries
Loaded chunk_016.pkl, combined dictionary now has 8000 e

Later I realized the issue with the UniProt accession 645345 and I want to fix this in the lookup table. (Doing it after because it is single issue and rather just modify what I collected and not have to run all the look-ups again. I embedded this step in the conditional so that in case authors address things later so that `645345` isn't there and more consistent `Q5T1J5` is, this code won't add an unnecessary item to `lookup_dict`.)

In [3]:
if '645345' in lookup_dict:
    del lookup_dict['645345']
    lookup_dict['645345'] = 'CHCHD2P9' # additional special handling later will insure `645345` gets changed to `Q5T1J5` in resulting CSV

As it doesn't take up too much space (list 234K), store the lookup table in pickled form for having in case need for reference later. Or for re-running downstream steps without needing to make the lookup table again. 

In [5]:
import pickle
with open("look_up_dict_all_13769_Uniprot_ACCs.pkl", "wb") as f:
        pickle.dump(lookup_dict, f)

Because it didn't take up too much space, I stored the lookup table in pickled form for having in case need for reference later. Or fo re-running without needing to make again. So optional code below to handle those optons. (Otherwise, in the way the code is in the cell, the cell runs fine without doing anything in present form.)

In [ ]:
# if need to read in the GIANT lookup dictonary
'''
import pickle
with open("look_up_dict_all_13769_Uniprot_ACCs.pkl", "rb") as f:
        lookup_dict = pickle.load(f)
len(list(lookup_dict.keys()))
'''

# if need to save the GIANT lookup dictionary again for some reason
'''
import pickle
with open("look_up_dict_all_13769_Uniprot_ACCs.pkl", "wb") as f:
        pickle.dump(lookup_dict, f)
'''

### Now use lookup table to fix and balance the two columns, `'Uniprot_ACCs'` & `'genenames'`

This is a step towards making a better CSV that adheres close to the author-provided one but follows better data management practices to make it easier to use Pandas `explode()` the way I had been when I though the two columns were in-order matched and I had not realized the extent of special cases buried in the data.

Also will do some accounting along the way so I can have more numbers to relate what was going on in original CSV. One aspect of this will be to assess how many special cases there are and make it easy to find them more easily.

In [5]:
import pickle
with open("look_up_dict_all_13769_Uniprot_ACCs.pkl", "rb") as f:
        lookup_dict = pickle.load(f)
len(list(lookup_dict.keys()))

13769

In [7]:
# now using the `lookup_dict` go through `rd_df` dataframe row by row and 
# replace the column Uniprot_ACCs and genenames with in-order matched, balanced
# versions and do accounting on them to track things along the way.
# The tracking dataframe that will result will be separate from the main one 
# because I need that to be good to save and replace the author-provided `hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922.csv`
fixed_df = rd_df.copy()
tracker_results_df = rd_df.copy()
class Tracker:
    def __init__(self):
        self.tracking_list = []

def make_ordered_fix_and_collect_info_about_balance(row, tracker):
    # for each row iterate on the identfiers in the column Uniprot_ACCs and 
    # account for the corresponding genename value in the genenames column. 
    # THIS WILL PRODUCED SAME NUMBER IN EACH COLUMN - so they are BALANCED in 
    # new version.
    # Do some tracking for accounting, too:
    # What is left? If anything is left then `genenames_had_unaccounted_for`
    # Also note if matching originally. Or if even balanced originally. (Note
    # that 'balanced originally' will take into account what I know about some 
    # Uniprot accensions corresponding to two genenames to produce the ones like 
    # `SLX1A; SLX1B` or `HSFY1; HSFY2` that have the semi-colon between the two 
    # related genemaes for the single Uniprot_ACC.
    # Or if one of the semi-colons is involved, etc.
    new_Uniprot_ACCs_list = []
    new_genenames_list = []
    original_ACCs_string = row['Uniprot_ACCs']
    original_ACCs_as_list = row['Uniprot_ACCs'].split()
    original_genenames_string = row['genenames']
    original_genenames_as_list = row['genenames'].split()
    # if original `genenames` has semi-colons, fix list based on what I had seen
    # in analysis earlier in this notebook, so what looks like two separated by
    # a `; ` are one without a space at all in the list. This is necessary to 
    # check the number of items balanced in each column later for 
    # 'balanced_originally' assessnent
    if ';' in original_genenames_string:
        # then fix the list
        adjusted_original_genenames_as_list = []
        i = 0
        while i < len(original_genenames_as_list):
            if original_genenames_as_list[i].endswith(';'):
                new_string = original_genenames_as_list[i] + original_genenames_as_list[i+1]
                adjusted_original_genenames_as_list.append(new_string)
                i += 2  # Skip the next element because already merged it into one before it to make one genename text string corresponding to the ID for a specific single Uniprot_ACC; for checking for balance
            else:
                adjusted_original_genenames_as_list.append(original_genenames_as_list[i])
                i += 1
        original_genenames_as_list = adjusted_original_genenames_as_list # now set the list to the fixed one
    what_remains = original_genenames_string # will be used to see if any genenames content not accounted for by content in Uniprot_ACCs column
    matching_originally = False # set this and only change if established below
    balanced_originally = False # set this and only change if established below
    involves_semicolon = False # set this and only change if established below
    SPECIAL_involved = False # set this and only change if established below
    genenames_had_unaccounted_for = False # set this and only change if established below
    what_genenames_had_unaccounted_for = "NOTHING" # set this and only change if established below

    # Sort the original_ACCs_as_list based on the length of their corresponding genenames
    # in lookup_dict (longest first)
    sorted_original_ACCs = sorted(
        original_ACCs_as_list,
        key=lambda x: len(lookup_dict[x]),
        reverse=True
    )
    # Now process in order of longest genename to shortest so for related genes like `TBL1X` and `TBL1XR1`, I don't end up removing `TBL1X` from `TBL1XR1` and just leave `R1` when making `what_remains`.
    for original_ACC in sorted_original_ACCs:
        if original_ACC == '645345': # handle this special case I noted where Uniprot_ACCs doesn't match a UniProt accessions; and this way if authors ever fix this won't happen and won't affect anything
            new_Uniprot_ACCs_list.append('Q5T1J5')
        else:
            new_Uniprot_ACCs_list.append(original_ACC) # TYPICAL FOR ALL EXCEPT ONE!
        matched_genename = lookup_dict[original_ACC]
        new_genenames_list.append(matched_genename.replace(" ","")) # the replace will remove the space after the semi-colon for any that have that; maybe I could have done it when I made the lookup table but I had decided at the time to keep the option to adhere closer to the orginal author-provided CSV representation but now realize when 'exploding' the contents of the columns to have one identifier pairing per row, the space after the semi-colon will be an issue in the way I did it based on my earlier assessment of the author-provided CSV. On the plus side of leaving it in, when I remove the matching text from the text in the genenames column to see what may remain unaccounted for based on the IDs in the Uniprot_ACCs columns, I don't have to specially add back in the space to match the original form.
        # While doing this iterating, start removing the accounted for 
        # identifiers from the second column string to end up with what is 
        # unaccounted for in the genenames column by the matching Uniprot_ACCs.
        # Importantly, only replace the first occurrence of the exact genename; 
        # this and the sorting I did above will help with related genes like 
        # `TBL1X` and `TBL1XR1`.
        parts = what_remains.split()
        if lookup_dict[original_ACC] in parts:
            parts.remove(lookup_dict[original_ACC]) #want the one that comes up
            # from the lookup_dict matching so that ones that have semi-colon in 
            # them removes both of the two name ones at the same time; the one 
            # I put in `new_genenames_list` above had the space removed and so 
            # if used that one the semi-colon related ones won't get deleted 
            # from what_remains properly and end up being specified erroneously 
            # as remaining
            what_remains = " ".join(parts)
        # need special handling for the cases where there is a semi-colon in the
        # matched genename and note want the one that comes up from the 
        # lookup_dict matching so that ones that have semi-colon in them removes 
        # both of the two name ones at the same time; the one. I put in 
        # `new_genenames_list` above had the space removed and so if used that 
        # one the semi-colon related ones won't get deleted from what_remains 
        # properly and end up being specified erroneously as remaining.
        if ';' in matched_genename and matched_genename.count(';') == 1: # some have more than one semi-colon and mostly I'll handle hardcoding below
            # Handle the semicolon case by finding and removing the exact pair
            parts = what_remains.split()
            i = 0
            while i < len(parts)-1:
                if parts[i].endswith(';'):
                    potential_pair = parts[i] + ' ' + parts[i+1]
                    if potential_pair == matched_genename:
                        # Remove both parts of the pair
                        parts.pop(i)
                        parts.pop(i)  # don't increment i since we removed two items
                        break
                    else:
                        i += 1
                else:
                    i += 1
            what_remains = ' '.join(parts)
        elif original_ACC in ['O15482','P23610','P62805','P68431','Q71DI3','P62807','P62807','P0C0S8']:
            # handle some special cases where there's semi-colons so default below won't work and so few that it isn't worth working out programmatically
            # SET UP TO Handle several special cases I noted, so proper text gets removed and don't add extra SPECIAL case to result CSV:
            if original_ACC == 'O15482':
                matched_genename = 'TEX28; TEX28P1; TEX28P2' # ones with more than a single semi-colon. Just going to hardcode handling removal so not tagged as unaccounted for.
            if original_ACC == 'P23610':
                matched_genename = 'F8A1; F8A2; F8A3'
            if original_ACC == 'P62805':
                matched_genename = 'H4C1; H4C2; H4C3; H4C4; H4C5; H4C6; H4C8; H4C9; H4C11; H4C12; H4C13; H4C14; H4C15; H4C16'
            if original_ACC == 'P68431':
                matched_genename = 'H3C1; H3C2; H3C3; H3C4; H3C6; H3C7; H3C8; H3C10; H3C11; H3C12'
            if original_ACC == 'Q71DI3':
                matched_genename = 'H3C15; H3C14; H3C13'
            if original_ACC == 'P62807':
                matched_genename = 'H2BC4; H2BC6; H2BC7; H2BC8; H2BC10'
            if original_ACC == 'P0C0S8':
                matched_genename = 'H2AC11; H2AC13; H2AC15; H2AC16; H2AC17'
            # END SET UP FOR SEVERAL SPECIAL CASES WITH MANY SEMI-COLONS
            what_remains = what_remains.replace(matched_genename,'')
        else:
            # Handle single genename case (when there is no semi-colon)
            parts = what_remains.split()
            # SET UP TO Handle several special cases I noted, so proper text gets removed and don't add extra SPECIAL case to resulting CSV:
            if matched_genename == 'SLC66A1LP':
                matched_genename = 'SLC66A1L' # makes it match what author provided file had for that one; they used one of its few synonyms
            if matched_genename == 'PRP4K':
                matched_genename = 'PRPF4B' # makes it match what author provided file had for that one; they used one of its few synonyms
            if matched_genename == 'CFAP263':
                matched_genename = 'CCDC113' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CFAP337':
                matched_genename = 'WDR49' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FERRY3':
                matched_genename = 'C12orf4' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CFAP184':
                matched_genename = 'CCDC96' # makes it match what author provided file had for that one; they used its synonym
            # END SET UP FOR SEVERAL SPECIAL CASES
            if matched_genename in parts:
                parts.remove(matched_genename)
                what_remains = ' '.join(parts)

    # now decide if anything extra has to be added to new_genenames_string by
    # accounting for everything in original add adding what remains. Anything 
    # that gets added will need a 'SPECIAL_'-branded id placeholder added to the
    # new_Uniprot_ACCs_list so that things remain balanced
    if what_remains.strip():
        genenames_had_unaccounted_for = True # set this for accounting
        what_genenames_had_unaccounted_for = what_remains.strip() # set this for accounting
        # now iterate on what splitting that string at the spaces returns and 
        # make a corresponding 'SPECIAL_'-labeled entry placeholder in the 
        unaccounted_genenames_list = what_genenames_had_unaccounted_for.split()
        for unaccounted_gename in unaccounted_genenames_list:
            new_Uniprot_ACCs_list.append('SPECIAL_unaccounted_gene') # NOTE, I used iterating on running this function & finding this occurence in the data to progressively add handling two types of 'special cases' (one type being those with threee or more semi-colons and the other being where the author-provided file use a synonyms) and ultimately end up with NO instances of `SPECIAL_unaccounted_gene` being necessary in my fixed in-order matched, balanced verison of the data.
            new_genenames_list.append(unaccounted_gename)
    assert (len(new_Uniprot_ACCs_list) == len(new_genenames_list)) #sanity 
    # check; they definitely should balance still
    new_Uniprot_ACCs_string = " ".join(new_Uniprot_ACCs_list)
    new_genenames_string = " ".join(new_genenames_list)
    # Now that done making in-order matched & balanced content, assign the 
    # appropriate text for those two columns to be returned when row returned at 
    # end of this entire function (before getting to end of this function 
    # though, will do some accounting)
    row['Uniprot_ACCs'] = new_Uniprot_ACCs_string
    row['genenames'] = new_genenames_string

    # Check original state for collecting some details
    '''
    for idx,oa in enumerate(original_ACCs_as_list):
        if original_genenames_as_list[idx] != lookup_dict[oa]:
            matching_originally = False
            break

    matching_originally = all(
        original_genenames_as_list[i] == lookup_dict[oa]
        for i, oa in enumerate(original_ACCs_as_list)
    )
    '''
    if len(original_ACCs_as_list) == len(original_genenames_as_list):
        balanced_originally = True
        # Only worth checking matching if balanced because otherwise get out `IndexError: list index out of range` as try to check `original_genenames_as_list[i]`
        matching_originally = all(
            original_genenames_as_list[i] == lookup_dict[oa]
            for i, oa in enumerate(original_ACCs_as_list)
        )
    if ';' in row['genenames']:
        involves_semicolon = True
    # Plus check the new state as part of the information being collected; the rows in the tracker will correspond to the fixed CSV to be made and so it will be pertinent to the corresponding row and I don't want to pollute the one to be fixed
    if any('SPECIAL' in item for item in new_Uniprot_ACCs_list + new_genenames_list):
        SPECIAL_involved =True
    # want to track, 'matching_originally','balanced originally','involves_semicolon','SPECIAL_involved','genenames_had_unaccounted_for','what_genenames_had_unaccounted_for'
    # This way later can use count of the numbers there to easy to see what is going on.
    tracker.tracking_list.append(
                            {
                            'matching_originally':matching_originally, 
                            'balanced_originally':balanced_originally, 
                            'involves_semicolon': involves_semicolon,
                            'SPECIAL_involved': SPECIAL_involved,
                            'genenames_had_unaccounted_for': genenames_had_unaccounted_for,
                            'what_genenames_had_unaccounted_for': what_genenames_had_unaccounted_for
                            })  # Append to the class attribute list a dictionary
    return row

tracker = Tracker()
fixed_df = fixed_df.apply(make_ordered_fix_and_collect_info_about_balance, args=(tracker,), axis=1)
# Save this as CSV
fixed_df.to_csv("hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv",index = False)
# will add to the tracker df as columns the lists corresponding to the values of the list of dictionaries
#matching_originally_values = [d['matching_originally'] for d in tracker]
#balanced_originally_values = [d['balanced_originally'] for d in tracker]
#involves_semicolon_values = [d['involves_semicolon'] for d in tracker]
#SPECIAL_involved_values = [d['SPECIAL_involved'] for d in tracker]
#genenames_had_unaccounted_for_values = [d['genenames_had_unaccounted_for'] for d in tracker]
#what_genenames_had_unaccounted_for_values = [d['what_genenames_had_unaccounted_for'] for d in tracker]
#tracker_results_df['matching_originally'] = matching_originally_values
tracker_results_df['matching_originally'] = [d['matching_originally'] for d in tracker.tracking_list] 
tracker_results_df['balanced_originally'] = [d['balanced_originally'] for d in tracker.tracking_list] 
tracker_results_df['involves_semicolon'] = [d['involves_semicolon'] for d in tracker.tracking_list]
tracker_results_df['SPECIAL_involved'] = [d['involves_semicolon'] for d in tracker.tracking_list]
tracker_results_df['genenames_had_unaccounted_for'] = [d['genenames_had_unaccounted_for'] for d in tracker.tracking_list]
tracker_results_df['what_genenames_had_unaccounted_for'] = [d['what_genenames_had_unaccounted_for'] for d in tracker.tracking_list]

In [9]:
tracker_results_df.head()

,HuMAP3_ID,ComplexConfidence,Uniprot_ACCs,genenames,matching_originally,balanced_originally,involves_semicolon,SPECIAL_involved,genenames_had_unaccounted_for,what_genenames_had_unaccounted_for
0,huMAP3_00000.1,1,P20963 Q9NWV4 Q9UGQ2,CD247 CACFD1 CZIB,False,True,False,False,False,NOTHING
1,huMAP3_00001.1,1,O94887 Q9NQ92 Q9NWB1,FARP2 COPRS RBFOX1,True,True,False,False,False,NOTHING
2,huMAP3_00002.1,1,Q8N3D4 Q9Y3A4,RRP7A EHBP1L1,False,True,False,False,False,NOTHING
3,huMAP3_00003.1,1,O00429 Q5T2D2,DNM1L TREML2,True,True,False,False,False,NOTHING
4,huMAP3_00004.1,1,O95460 P21941 P78540 Q9H267 Q9H9C1,MATN4 MATN1 ARG2 VPS33B VIPAS39,True,True,False,False,False,NOTHING


I know from searching for 'SPECIAL_unaccounted' in the resulting CSV that my iterating on things and adding handling for special cases involving the use of synonym names and handling caseses with two or more semi-colons, i.e., three or more gene names for same UniProt accession, eliminated all those, so at this point `genenames_had_unaccounted_for` and	`what_genenames_had_unaccounted_for` are moot and can be removed from the tracker data.  
Let's verify that here and then remove those two tracking columns.

In [15]:
unf_count_df = tracker_results_df.copy().value_counts('genenames_had_unaccounted_for').reset_index()
unf_count_df.head()

,genenames_had_unaccounted_for,count
0,False,15326


In [16]:
tracker_results_df = tracker_results_df.drop(columns=['genenames_had_unaccounted_for', 'what_genenames_had_unaccounted_for'])
tracker_results_df.head()

,HuMAP3_ID,ComplexConfidence,Uniprot_ACCs,genenames,matching_originally,balanced_originally,involves_semicolon,SPECIAL_involved
0,huMAP3_00000.1,1,P20963 Q9NWV4 Q9UGQ2,CD247 CACFD1 CZIB,False,True,False,False
1,huMAP3_00001.1,1,O94887 Q9NQ92 Q9NWB1,FARP2 COPRS RBFOX1,True,True,False,False
2,huMAP3_00002.1,1,Q8N3D4 Q9Y3A4,RRP7A EHBP1L1,False,True,False,False
3,huMAP3_00003.1,1,O00429 Q5T2D2,DNM1L TREML2,True,True,False,False
4,huMAP3_00004.1,1,O95460 P21941 P78540 Q9H267 Q9H9C1,MATN4 MATN1 ARG2 VPS33B VIPAS39,True,True,False,False


Now use the 'tracker_results_df' that parallels the dataframe I used to make the resulting CSV to get some idea of numbers of what rows were originally in-order matchings & balanced, plus some details of the results I made to fix things to be closer to what will easiluy make tidy data using Pandas `explode()`:

In [12]:
mo_count_df = tracker_results_df.copy().value_counts('matching_originally').reset_index()
mo_count_df.head()

,matching_originally,count
0,True,9679
1,False,5647


In [13]:
mo_count_df = tracker_results_df.copy().value_counts('matching_originally', normalize=True).reset_index()
mo_count_df.head()

,matching_originally,proportion
0,True,0.631541
1,False,0.368459


So way more than half were matchcing the order in the two columns, `Uniprot_ACCs` & `genenames`, in the original CsV. In fact, about two-thirds matched, and so I wasn't totally clueless about not picking up on that earlier.

In [17]:
bal_count_df = tracker_results_df.copy().value_counts('balanced_originally').reset_index()
bal_count_df.head()

,balanced_originally,count
0,True,15192
1,False,134


I was intitially surprised that number is so low since I had originall noted 279 having different number of items in the same row, but that was before I accounted for the semi-colon with space use to specify multiple gene names for a single protein. I think that accounts for the discrepancy because here I was counting those as one and not just use space character to do the string splitting, like when I saw 279.

In [19]:
sc_count_df = tracker_results_df.copy().value_counts('involves_semicolon').reset_index()
sc_count_df.head()

,involves_semicolon,count
0,False,15149
1,True,177


So given the 134 not balanced originally, and what I just mentioned about those using semi-colons combined with a space to indicate related gene names for same UniProt accession identifier, the ones that incolve the semi-colon should somewhat account for the rest. Let's sum those up and see:

In [20]:
177 + 134 

311

That number is higher than 279 because just by chance some of those would balance when split on spaces and so I would have missed some, but it is in the same ball-park.

In my 'fixed' version of the CSV data what is the amount of 'SPECIAL' cases I identified?
(Note there were no cases of 'not_known' in there because while putting these last steps together to do this accounting, I went back and looked at the issue with UniProt accession `645345` and found I knew it had a UniProt accession & gene name I could assign with some special handling. So I eliminted any of that type with some additional handling.)

In [21]:
sp_count_df = tracker_results_df.copy().value_counts('SPECIAL_involved').reset_index()
sp_count_df.head()

,SPECIAL_involved,count
0,False,15149
1,True,177


In [22]:
sp_count_df = tracker_results_df.copy().value_counts('SPECIAL_involved', normalize=True).reset_index()
sp_count_df.head()

,SPECIAL_involved,proportion
0,False,0.988451
1,True,0.011549


So there's a fairly small fraction , barely above 1%, that are 'special' cases in there now.
And even if these are special there is matching information in both colunns in my 'fixed' CSV to keep things explodable and utimately tidy.

**Use `hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv` saved above the 'accounting' I just did.**

#### Next few cells (stored converted to 'raw') for development debugging of lookup table generating only; leave as found unless you are Wayne troubleshooting:

------

In [23]:
# for development; to keep kernel active
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()

.


KeyboardInterrupt: 